In [4]:
import numpy as np
import pandas as pd
import json
import panel as pn
from matplotlib import pyplot as plt
import hvplot.pandas
from PIL import Image
import os, os.path
from bokeh.models import HoverTool


In [5]:
# load data
f = 'sample6_cmec.json'
with open(f) as og_file:
    dat = json.load(og_file)

In [6]:
h = 'peak_season_cmec.json'
with open(h) as new_file:
    dat2 = json.load(new_file)

In [7]:
ps_df = pd.DataFrame.from_dict(dat2['RESULTS'], orient='index')
ps_df['Model'] = ps_df.index
ps_df = ps_df.reset_index()
ps_df = ps_df[['Model', 'peak season']]
ps_df['var_name'] = 'peak season'
ps_df

,Model,peak season,var_name
0,BCC-CSM2-MR,6,peak season
1,CSIRO-Mk3-6-0,-4,peak season
2,CanESM2,17,peak season
3,ERA5,14,peak season
4,MERRA2,11,peak season
5,MRI-ESM2-0,-3,peak season
6,NorESM1-M,1,peak season


In [8]:
# load images
imgs = []
path = "./images/"
for p in os.listdir(path):
    ext = os.path.splitext(p)[1]
    imgs.append(Image.open(os.path.join(path,p)))

# view one image
imgs[0].show()

In [9]:
# add image links to dataframe
# edit hyperlinks from github repo to match format below (add raw.githubusercontent.com, remove "blob" from path)

img_links = ['https://raw.githubusercontent.com/kristinchang3/PCMDI-Dong12-test/main/images/CSIRO-Mk3.png',
             'https://raw.githubusercontent.com/kristinchang3/PCMDI-Dong12-test/main/images/CSIRO-Mk3.png',
             'https://raw.githubusercontent.com/kristinchang3/PCMDI-Dong12-test/main/images/CanESM2.png',
             'https://raw.githubusercontent.com/kristinchang3/PCMDI-Dong12-test/main/images/ERA5.png',
             'https://raw.githubusercontent.com/kristinchang3/PCMDI-Dong12-test/main/images/MERRA2.png',
             'https://raw.githubusercontent.com/kristinchang3/PCMDI-Dong12-test/main/images/MRI-ESM2.png',
             'https://raw.githubusercontent.com/kristinchang3/PCMDI-Dong12-test/main/images/NorESM1.png', 
             ]

ps_df['img'] = img_links

# adjust pandas settings to view full column width
pd.set_option('max_colwidth', 1000)
ps_df

,Model,peak season,var_name,img
0,BCC-CSM2-MR,6,peak season,https://raw.githubusercontent.com/kristinchang3/PCMDI-Dong12-test/main/images/CSIRO-Mk3.png
1,CSIRO-Mk3-6-0,-4,peak season,https://raw.githubusercontent.com/kristinchang3/PCMDI-Dong12-test/main/images/CSIRO-Mk3.png
2,CanESM2,17,peak season,https://raw.githubusercontent.com/kristinchang3/PCMDI-Dong12-test/main/images/CanESM2.png
3,ERA5,14,peak season,https://raw.githubusercontent.com/kristinchang3/PCMDI-Dong12-test/main/images/ERA5.png
4,MERRA2,11,peak season,https://raw.githubusercontent.com/kristinchang3/PCMDI-Dong12-test/main/images/MERRA2.png
5,MRI-ESM2-0,-3,peak season,https://raw.githubusercontent.com/kristinchang3/PCMDI-Dong12-test/main/images/MRI-ESM2.png
6,NorESM1-M,1,peak season,https://raw.githubusercontent.com/kristinchang3/PCMDI-Dong12-test/main/images/NorESM1.png


In [28]:
# html for custom hover tool
hover = HoverTool(tooltips="""
    <div>
        <div>
            <img src="@img" width=300 style="float: left; margin: 0px 15px 15px 0px; border="2"></img>
        </div>
    </div>

""")

In [29]:
peak_plot01 = ps_df.hvplot.heatmap(y='var_name',
                       x='Model',
                       C='peak season',
                       hover_cols = ['img'],
                       tools = [hover],
                       height = 200,
                       width=800,
                       colorbar=True,
                       xaxis='top',
                       cmap='bwr').opts(xrotation=45)
peak_plot01

:HeatMap   [Model,var_name]   (peak season,img)

In [25]:
hvplot.save(peak_plot01, 'peak_plot04.html')

In [7]:
# create dfs for each model results

BCC_CSM2_MR = pd.DataFrame.from_dict(dat['RESULTS']['BCC-CSM2-MR']['area (km2)']['NDJFM'], orient='index')
CanESM2 = pd.DataFrame.from_dict(dat['RESULTS']['CanESM2']['area (km2)']['NDJFM'], orient='index')
CCSM4 = pd.DataFrame.from_dict(dat['RESULTS']['CCSM4']['area (km2)']['NDJFM'], orient='index')
CSIRO_Mk3_6_0 = pd.DataFrame.from_dict(dat['RESULTS']['CSIRO-Mk3-6-0']['area (km2)']['NDJFM'], orient='index')
NorESM1_M = pd.DataFrame.from_dict(dat['RESULTS']['NorESM1-M']['area (km2)']['NDJFM'], orient='index')

In [8]:
# create list of vars
vars = np.array(pd.DataFrame(dat['RESULTS']).index)

# create list of models
models = pd.DataFrame(dat['RESULTS']).columns.values

In [9]:
# create array of dfs with just model results
dfs = []
for i in models:
    for j in vars:
        df = pd.DataFrame(data=(dat['RESULTS'][i][j]['NDJFM']), index=[0])
        df['Model'] = i
        df['variable'] = j
        dfs.append(df)

# create single df and organize
results = pd.concat(dfs).reset_index()
cols = ['Model', 'variable', 'mean bias', 'ratio of STD', 'normalized bias']
results = results[cols]


In [10]:
results

,Model,variable,mean bias,ratio of STD,normalized bias
0,BCC-CSM2-MR,latitude(degN),0.58,1.12,0.053241
1,BCC-CSM2-MR,longitude(degE),2.80,1.03,0.068014
2,BCC-CSM2-MR,area (km2),90721.18,1.07,0.076336
3,BCC-CSM2-MR,length(GCD),-0.03,1.03,-0.015897
4,BCC-CSM2-MR,width(GCD),-0.12,0.99,-0.015996
5,CanESM2,latitude(degN),0.51,1.04,0.046425
6,CanESM2,longitude(degE),0.43,1.01,0.010460
7,CanESM2,area (km2),-67473.88,0.87,-0.056775
8,CanESM2,length(GCD),-0.44,0.82,-0.241871
9,CanESM2,width(GCD),-1.44,0.81,-0.185825


In [14]:
results['img'] = 'https://github.com/kristinchang3/PCMDI-Dong12-test/blob/main/charts/Hist01.png'
results.head()

,Model,variable,mean bias,ratio of STD,normalized bias,img
0,BCC-CSM2-MR,latitude(degN),0.58,1.12,0.053241,https://github.com/kristinchang3/PCMDI-Dong12-...
1,BCC-CSM2-MR,longitude(degE),2.80,1.03,0.068014,https://github.com/kristinchang3/PCMDI-Dong12-...
2,BCC-CSM2-MR,area (km2),90721.18,1.07,0.076336,https://github.com/kristinchang3/PCMDI-Dong12-...
3,BCC-CSM2-MR,length(GCD),-0.03,1.03,-0.015897,https://github.com/kristinchang3/PCMDI-Dong12-...
4,BCC-CSM2-MR,width(GCD),-0.12,0.99,-0.015996,https://github.com/kristinchang3/PCMDI-Dong12-...


In [15]:
hover = HoverTool(tooltips="""
    <div>
        <div>
            <img src="@img" width=100 style="float: left; margin: 0px 15px 15px 0px; border="2"></img>
        </div>
    </div>

""")

In [16]:
# generate interactive plot
portrait01 = results.hvplot.heatmap(x='variable',
                       y='Model',
                       C='normalized bias',
                       hover_cols = ['img'],
                       tools = [hover],
                       height = 500,
                       width=600,
                       colorbar=True,
                       xaxis='top',
                       cmap='bwr').opts(xrotation=45)
portrait01


:HeatMap   [variable,Model]   (normalized bias,img)

In [ ]:
hvplot.save(portrait01, 'portrait01.html')

In [ ]:
results_test = results.loc[results['Model']=='BCC-CSM2-MR']
results_test

,Model,variable,mean bias,ratio of STD,normalized bias,img
0,BCC-CSM2-MR,latitude(degN),0.58,1.12,0.053241,https://github.com/kristinchang3/PCMDI-Dong12-...
1,BCC-CSM2-MR,longitude(degE),2.80,1.03,0.068014,https://github.com/kristinchang3/PCMDI-Dong12-...
2,BCC-CSM2-MR,area (km2),90721.18,1.07,0.076336,https://github.com/kristinchang3/PCMDI-Dong12-...
3,BCC-CSM2-MR,length(GCD),-0.03,1.03,-0.015897,https://github.com/kristinchang3/PCMDI-Dong12-...
4,BCC-CSM2-MR,width(GCD),-0.12,0.99,-0.015996,https://github.com/kristinchang3/PCMDI-Dong12-...


In [ ]:
results_test.hvplot.heatmap(x='variable',
                       y='Model',
                       C='normalized bias',
                       hover_cols = ['img'],
                       tools = [hover],
                       height = 500,
                       width=600,
                       colorbar=True,
                       xaxis='top',
                       cmap='bwr').opts(xrotation=45)

:HeatMap   [variable,Model]   (normalized bias,img)

In [ ]:
results['Model'].unique()

array(['BCC-CSM2-MR', 'CanESM2', 'CCSM4', 'CSIRO-Mk3-6-0', 'NorESM1-M',
       'MRI-ESM2-0'], dtype=object)

In [ ]:
d = pd.DataFrame({'peak season':[6, 17, 4, 1, 3], 'b':['BCC-CSM2-MR', 'CanESM2', 'CSIRO-Mk3-6-0', 'NorESM1-M',
       'MRI-ESM2-0']})

In [ ]:
d['var'] = 'peak season'
d

,peak season,b,var
0,6,BCC-CSM2-MR,peak season
1,17,CanESM2,peak season
2,4,CSIRO-Mk3-6-0,peak season
3,1,NorESM1-M,peak season
4,3,MRI-ESM2-0,peak season


In [ ]:
d.hvplot.heatmap(y='var',
                       x='b',
                       C='peak season',
                       height =200,
                       width=500,
                       colorbar=True,
                       xaxis='top',
                       cmap='bwr').opts(xrotation=45)

:HeatMap   [b,var]   (peak season)

In [ ]:
import panel as pn
pn.extension()

In [ ]:
livepage = pn.Row(portrait01).servable()
livepage

Row
    [0] HoloViews(HeatMap)